In [6]:
import os
os.getcwd()

'C:\\Users\\tbgoo\\Documents\\Python Scripts\\URP 2020'

In [1]:
from get_data import data as dat

file = ("/users/tbgoo/Desktop/Guitar Samples/Dark Guitar String.wav")
window = 'blackman'
M = 3001 # size of window
t = -80 #threshold
N = 4096 #size of fft
Ns = 512 #size of fft used in synthesis
freqDevOffset=20
freqDevSlope=0.1
maxnSines = 8
minSineDur=.01

sample_rate, freaks, ffts, mags = dat(file, window, M, t, N, Ns, freqDevOffset, freqDevSlope, maxnSines, minSineDur)

C:\Users\tbgoo\anaconda\lib\site-packages\scipy\io\wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


In [ ]:
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np

# graph code
nyquist = sample_rate/2
binsize = (nyquist)/(N/2)
fscale = np.arange(0,nyquist,step=binsize)
harmind = np.arange(80/binsize,nyquist,step=80/binsize, dtype=int)
numHarms = 20

def plot_func(i):
    hfreq = np.arange(freaks[i,0],nyquist,step=freaks[i,0])
    Ifreak = int((numHarms*80)/binsize)
    #plt.figure(figsize = (16,7))
    #plt.plot(fscale[0:Ifreak],ffts[i,0:Ifreak])
    #plt.plot(freaks[i,0:numHarms],mags[i,0:numHarms],'o')
    #plt.plot(freaks[i,0:numHarms],(mags[i,0:numHarms]-mags[i,0]),'o')
    plt.plot(mags[:,0])
    plt.show()

interact(plot_func, i = widgets.IntSlider(value=100,
                                          min=1,
                                          max=2300))

In [2]:
def sineSubtraction(x, N, H, sfreq, smag, sphase, fs):
	"""
	Subtract sinusoids from a sound
	x: input sound, N: fft-size, H: hop-size
	sfreq, smag, sphase: sinusoidal frequencies, magnitudes and phases
	returns xr: residual sound
	"""

	hN = N//2                                          # half of fft size
	x = np.append(np.zeros(hN),x)                      # add zeros at beginning to center first window at sample 0
	x = np.append(x,np.zeros(hN))                      # add zeros at the end to analyze last sample
	bh = blackmanharris(N)                             # blackman harris window
	w = bh/ sum(bh)                                    # normalize window
	sw = np.zeros(N)                                   # initialize synthesis window
	sw[hN-H:hN+H] = triang(2*H) / w[hN-H:hN+H]         # synthesis window
	L = sfreq.shape[0]                                 # number of frames, this works if no sines
	xr = np.zeros(x.size)                              # initialize output array
	pin = 0
	for l in range(L):
		xw = x[pin:pin+N]*w                              # window the input sound
		X = fft(fftshift(xw))                            # compute FFT
		Yh = UF_C.genSpecSines(N*sfreq[l,:]/fs, smag[l,:], sphase[l,:], N)   # generate spec sines
		Xr = X-Yh                                        # subtract sines from original spectrum
		xrw = np.real(fftshift(ifft(Xr)))                # inverse FFT
		xr[pin:pin+N] += xrw*sw                          # overlap-add
		pin += H                                         # advance sound pointer
	xr = np.delete(xr, range(hN))                      # delete half of first window which was added in stftAnal
	xr = np.delete(xr, range(xr.size-hN, xr.size))     # delete half of last window which was added in stftAnal
	return xr

In [4]:
import numpy as np
from scipy.signal import blackmanharris, triang
from scipy.fftpack import ifft, fftshift
import math

def sineModelSynth(xtfreq, xtmag, xtphase, N, H, fs):
	"""
	Synthesis of a sound using the sinusoidal model
	tfreq,tmag,tphase: frequencies, magnitudes and phases of sinusoids
	N: synthesis FFT size, H: hop size, fs: sampling rate
	returns y: output array sound
	"""
	
	hN = N//2                                               # half of FFT size for synthesis
	L = tfreq.shape[0]                                      # number of frames
	pout = 0                                                # initialize output sound pointer         
	ysize = H*(L+3)                                         # output sound size
	y = np.zeros(ysize)                                     # initialize output array
	sw = np.zeros(N)                                        # initialize synthesis window
	ow = triang(2*H)                                        # triangular window
	sw[hN-H:hN+H] = ow                                      # add triangular window
	bh = blackmanharris(N)                                  # blackmanharris window
	bh = bh / sum(bh)                                       # normalized blackmanharris window
	sw[hN-H:hN+H] = sw[hN-H:hN+H]/bh[hN-H:hN+H]             # normalized synthesis window
	lastytfreq = tfreq[0,:]                                 # initialize synthesis frequencies
	ytphase = 2*np.pi*np.random.rand(tfreq[0,:].size)       # initialize synthesis phases 
	for l in range(L):                                      # iterate over all frames
		if (tphase.size > 0):                                 # if no phases generate them
			ytphase = tphase[l,:] 
		else:
			ytphase += (np.pi*(lastytfreq+tfreq[l,:])/fs)*H     # propagate phases
		Y = UF.genSpecSines(tfreq[l,:], tmag[l,:], ytphase, N, fs)  # generate sines in the spectrum         
		lastytfreq = tfreq[l,:]                               # save frequency for phase propagation
		ytphase = ytphase % (2*np.pi)                         # make phase inside 2*pi
		yw = np.real(fftshift(ifft(Y)))                       # compute inverse FFT
		y[pout:pout+N] += sw*yw                               # overlap-add and apply a synthesis window
		pout += H                                             # advance sound pointer
	y = np.delete(y, range(hN))                             # delete half of first window
	y = np.delete(y, range(y.size-hN, y.size))              # delete half of the last window 
	return y
	

In [11]:
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

nyquist = sample_rate/2
binsize = (nyquist)/(N/2)
fscale = np.arange(0,nyquist,step=binsize)
harmind = np.arange(80/binsize,nyquist,step=80/binsize, dtype=int)
numHarms = 20

def closest2(lowF,hiF):
    lowInd = abs(fscale - lowF).argmin()
    hiInd = abs(fscale - hiF).argmin()
    return lowInd, hiInd

def binInterp(mX, lowInd, ind):
    """
    Interpolate peak values using parabolic interpolation
    mX, pX: magnitude and phase spectrum, ploc: locations of peaks
    returns iploc, ipmag, ipphase: interpolated peak location, magnitude and phase values
    """
    ploc = lowInd+ind
    val = mX[ploc]                                          # magnitude of peak bin
    if(ind==0):
        lval = mX[ploc+1]                                       # magnitude of bin at left
        rval = mX[ploc+1]                                       # magnitude of bin at right
    elif(ind==1):
        lval = mX[ploc-1]                                       # magnitude of bin at left
        rval = mX[ploc+1]                                       # magnitude of bin at right
    elif(ind==2):
        lval = mX[ploc-1]                                       # magnitude of bin at left
        rval = mX[ploc-1]
        
    iploc = ploc + 0.5*(lval-rval)/(lval-2*val+rval)        # center of parabola
    ipmag = val - 0.25*(lval-rval)*(iploc-ploc)             # magnitude of peaks
    return iploc, ipmag
"""
i = 1064
f0 = freaks[i,0]
hfreq = np.arange(f0,nyquist,step=f0)
harm = hfreq[10] # overtone 18 determined by f0
peaks = freaks[i,:]
lowF = harm-(f0/2)
hiF = harm+(f0/2)
peaks = peaks[peaks>=lowF]
peaks = peaks[peaks<=hiF]
if(peaks.shape[0]>0):
    peak = peaks[abs(peaks-harm).argmin()]
else:
    lowInd,hiInd = closest2(lowF,hiF)
    ind = ffts[i,lowInd:hiInd].argmax()
    iploc, ipmag = binInterp(ffts[i,:],lowInd,ind)
    ipfreq = sample_rate*iploc/float(N)                            # convert peak locations to Hertz
    """

'\ni = 1064\nf0 = freaks[i,0]\nhfreq = np.arange(f0,nyquist,step=f0)\nharm = hfreq[10] # overtone 18 determined by f0\npeaks = freaks[i,:]\nlowF = harm-(f0/2)\nhiF = harm+(f0/2)\npeaks = peaks[peaks>=lowF]\npeaks = peaks[peaks<=hiF]\nif(peaks.shape[0]>0):\n    peak = peaks[abs(peaks-harm).argmin()]\nelse:\n    lowInd,hiInd = closest2(lowF,hiF)\n    ind = ffts[i,lowInd:hiInd].argmax()\n    iploc, ipmag = binInterp(ffts[i,:],lowInd,ind)\n    ipfreq = sample_rate*iploc/float(N)                            # convert peak locations to Hertz\n    '

In [81]:
freqs = np.array([])

numags = np.array([])
for i in range(mags.shape[0]):
    f0 = freaks[i,0]
    hfreq = np.arange(f0,nyquist,step=f0)
    harm = hfreq[10] # overtone 18 determined by f0
    peaks = freaks[i,:]
    Pmags = mags[i,:]
    lowF = harm-(f0/2)
    hiF = harm+(f0/2)
    peaks = peaks[peaks>=lowF]
    peaks = peaks[peaks<=hiF]
    if(peaks.shape[0]>0):
        ind = abs(peaks-harm).argmin()
        freqs = np.append(freqs,peaks[ind])
        numags = np.append(numags,Pmags[ind])
    else:
        lowInd,hiInd = closest2(lowF,hiF)
        ind = ffts[i,lowInd:hiInd].argmax()
        iploc, ipmag = binInterp(ffts[i,:],lowInd,ind)
        ipfreq = sample_rate*iploc/float(N)                            # convert peak locations to Hertz
        freqs = np.append(freqs, ipfreq)
        numags = np.append(numags, ipmag)

UnboundLocalError: local variable 'lval' referenced before assignment

In [36]:
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

def plot_func(i):
    numval = xtmag[:,i].shape[0]
    x = np.arange(0,numval)
    plt.plot(xtmag[:,i])
    plt.plot(x*slopes[i]+yint[i])
    plt.show()
interact(plot_func, i = widgets.IntSlider(value=0,
                                               min=0,
                                               max=19))

A Jupyter Widget

<function __main__.plot_func>

In [28]:
import scipy.optimize as sci
from scipy import polyfit
from scipy import optimize

x = np.arange(50,numfft)
lines = np.zeros(shape=(3))

slopes = np.zeros(shape=0)
yint = np.zeros(shape=0)
numsamps = 5

for i in range(numsamps):
        vals = xtmag[:,i]
        ind = np.where(vals>-120)[0]
        vols = vals[ind]
        var = polyfit(ind,vols,1)
        slopes = np.append(slopes,var[0])
        yint = np.append(yint,var[1])


In [2]:
import numpy as np
from test_func import test as t
x = np.array([10,2,4,7,1])

array([ 1,  2,  4,  7, 10])

In [14]:
xr = sineSubtraction(audio_data, Ns, H, xtfreq, xtmag, xtphase, sample_rate)

ValueError: operands could not be broadcast together with shapes (2048,) (512,) 